Contents:
 1. Run MR-APSS at IV threshold 5e-05 (default) and evaluate the importance of MR-APSS for accounting for pleiotropy, sample structure, and selection bias.
 2. Run MR-APSS by changing the IV thresholds to show that MR-APSS is insensitive to IV thresholds.

# The function for calling MRAPSS() from MRAPSS package

In [3]:
run_APSS_func <- function(clumped=NULL,
                          exposure=NULL,
                          outcome=NULL,
                          C = diag(2),
                          Omega = matrix(0, 2, 2),
                          IV.Threshold = 5e-08, # IV selection threshold
                          Threshold = 5e-08,    # threshold for correcting for selection bias
                          Cor.SelectionBias=T){
  
 
  res=NULL
  
  for(i in 1:length(Threshold)){
    
    start = proc.time()
    if(!is.null(clumped)){
      
      test = subset(clumped, pval.exp <= IV.Threshold[i])
      if(nrow(test) < 4 ) next
      test$Threshold = Threshold[i]
      cat("IV selection threshold:", IV.Threshold[i] ,"\n")
      
      
      MRres = try(MRAPSS::MRAPSS(test,
                                 exposure=exposure,
                                 outcome=outcome,
                                 C= C,
                                 Omega= Omega,
                                 Cor.SelectionBias = Cor.SelectionBias))
 
      print(proc.time()-start)
      
      if(inherits(MRres, 'try-error')) {
        MRres=NULL
      }
    }
    
    
    res0 = data.frame(exposure = MRres$exposure,
                      outcome = MRres$outcome,
                      nSNP = nrow(MRres$MRdat),
                      pi0 = MRres$pi0,
                      nvalid = nrow(MRres$MRdat)*MRres$pi0,
                      sigma.sq= MRres$sigma.sq,
                      tau.sq= MRres$tau.sq,
                      beta = MRres$beta,
                      se = MRres$beta.se,
                      pval= MRres$pvalue,
                      method = MRres$method,
                      Threshold = Threshold[i],
                      IVstrength = MRres$IVsignal.sum
    )
    
    if(nrow(res0)!=0){
      res0$IV.Threshold = IV.Threshold[i]
    }
    
    res = rbind(res, res0)
    
  }
  
  return(res)
  
}



# Run MR-APSS at IV threshold 5e-05 (default)

In [3]:
# All the results will be saved to the file "NC_MRAPSS.MRres"
start = proc.time()
ts1 = c("AD", "ASD", "Daytime_Sleepiness", "Height_UKB",  "Intelligence", "RA",      
        "T2D", "Alcohol", "BMI", "Depression", "IBD", "MDD", "SCZ", "Angina", 
        "CAD", "HBP", "Income", "NEB", "Smoking", "Urate", "Anorexia", 
        "CD", "Height_GIANT", "Insomnia", "Neuroticism", "SWB")

ts2 = c("Hair_Light_Brown", "Hair_Dark_Brown",  "Hair_Black",    "Hair_Blonde",  "Tanning")

IV.Threshold = 5e-05

for( exposure in ts1){
  
  for( outcome in ts2){
    
    cat("Pair: ", exposure,"~", outcome,"\n")
    # read in GWAS summary data for IVs
    clumped = try(read.table(paste0("./MRdat/", exposure,"~",outcome), header = T))
    
    # read in background parameters Omega and C
    C = try(as.matrix(read.table(paste0("./pairs_bg_paras/",exposure, "~", outcome,"_C"),
                                 header = F)))
    
    C = matrix(C[nrow(C), ], 2,2)
    
    Omega = try(as.matrix(read.table(paste0("./pairs_bg_paras/", exposure, "~", outcome,"_Omega"),
                                     header = F)))
      
    Omega = matrix(Omega[nrow(Omega), ], 2, 2)
    
    if(inherits(clumped , 'try-error')) clumped = NULL
    if(inherits(C, 'try-error')) next
    if(inherits(Omega, 'try-error')) next
    if(nrow(clumped) < 4 ) next
    
    # The p-value threshold for selection bias correction
    Threshold =  ifelse(IV.Threshold==5e-05, unique(clumped$Threshold), IV.Threshold)
    
    # MR-APSS
    cat("Run MR-APSS ... \n")
    res = run_APSS_func(clumped = clumped,
                        exposure = exposure,
                        outcome = outcome,
                        C = C,
                        Omega=Omega,
                        IV.Threshold = IV.Threshold,
                        Threshold = Threshold,
                        Cor.SelectionBias = T)
    
    # MR-APSS(Omega=0)
    cat("Run MR-APSS (Omega = 0), i.e. MR-APSS not accounting for polygenicity and pleiotropy. ... \n")
    res_Omega0 = run_APSS_func(clumped = clumped,
                               exposure = exposure,
                               outcome = outcome,
                               C = C,
                               Omega=matrix(0, 2, 2),
                               IV.Threshold = IV.Threshold,
                               Threshold = Threshold,
                               Cor.SelectionBias = T)
    res_Omega0$method = "MR-APSS(Omega=0)"
     
     # MR-APSS(C=I)
     cat("Run MR-APSS (C = I), i.e. MR-APSS not accounting for sample structure ... \n")
     res_CI = run_APSS_func(clumped = clumped,
                                exposure = exposure,
                                outcome = outcome,
                                C = diag(2),
                                Omega = Omega,
                                IV.Threshold = IV.Threshold,
                                Threshold = Threshold,
                                Cor.SelectionBias = T)
     res_CI$method = "MR-APSS(C=I)"
     
     # MR-APSS not accounting for selection bias
     cat("Run MR-APSS not accounting for selection bias ... \n")
     res_selec0 = run_APSS_func(clumped = clumped,
                            exposure = exposure,
                            outcome = outcome,
                            C = C,
                            Omega = Omega,
                            IV.Threshold = IV.Threshold,
                            Threshold = Threshold,
                            Cor.SelectionBias = F)
     res_selec0$method = "MR-APSS(Cor.SelectionBia=F)"
     
     # saving resuts
     write.table(res, "NC_MRAPSS.MRres", quote=F, col.names = F, append = T,row.names = F)
     
     write.table(res_Omega0, "NC_MRAPSS.MRres", quote=F, col.names = F, append = T,row.names = F)
     
     write.table(res_CI, "NC_MRAPSS.MRres", quote=F, col.names = F, append = T,row.names = F)
     
     write.table(res_selec0, "NC_MRAPSS.MRres", quote=F, col.names = F, append = T,row.names = F)
     
}
  
}

print(proc.time()-start)


Pair:  AD ~ Hair_Light_Brown 
Run MR-APSS ... 
IV selection threshold: 5e-05 
***********************************************************
MR test results of  AD  on  Hair_Light_Brown : 
MR-APSS: beta =  -0.0067 , beta.se =  0.0187 , p-value =  7.1987e-01 . 
Total NO. of IVs=  69 , NO. of valid IVs with foreground signals:  17.08942 . 
***********************************************************
   user  system elapsed 
  2.062   3.378   1.493 
Run MR-APSS (Omega = 0), i.e. MR-APSS not accounting for polygenicity and pleiotropy. ... 
IV selection threshold: 5e-05 
***********************************************************
MR test results of  AD  on  Hair_Light_Brown : 
MR-APSS: beta =  -0.0058 , beta.se =  0.0175 , p-value =  7.4077e-01 . 
Total NO. of IVs=  69 , NO. of valid IVs with foreground signals:  17.68984 . 
***********************************************************
   user  system elapsed 
  1.095   0.728   0.911 
Run MR-APSS (C = I), i.e. MR-APSS not accounting for sample st

Threshold = 1, the model will not correct for selection bias



***********************************************************
MR test results of  AD  on  Hair_Light_Brown : 
MR-APSS: beta =  -0.001 , beta.se =  0.0116 , p-value =  9.2870e-01 . 
Total NO. of IVs=  69 , NO. of valid IVs with foreground signals:  68.9931 . 
***********************************************************
   user  system elapsed 
 30.622  19.263  24.949 
Pair:  AD ~ Hair_Dark_Brown 
Run MR-APSS ... 
IV selection threshold: 5e-05 
***********************************************************
MR test results of  AD  on  Hair_Dark_Brown : 
MR-APSS: beta =  0 , beta.se =  0.0044 , p-value =  9.9758e-01 . 
Total NO. of IVs=  69 , NO. of valid IVs with foreground signals:  17.04725 . 
***********************************************************
   user  system elapsed 
   0.93    0.53    0.73 
Run MR-APSS (Omega = 0), i.e. MR-APSS not accounting for polygenicity and pleiotropy. ... 
IV selection threshold: 5e-05 
***********************************************************
MR test resu

Threshold = 1, the model will not correct for selection bias



***********************************************************
MR test results of  AD  on  Hair_Dark_Brown : 
MR-APSS: beta =  0.0027 , beta.se =  0.0133 , p-value =  8.3785e-01 . 
Total NO. of IVs=  69 , NO. of valid IVs with foreground signals:  68.9931 . 
***********************************************************
   user  system elapsed 
  0.299   0.191   0.245 
Pair:  AD ~ Hair_Black 
Run MR-APSS ... 
IV selection threshold: 5e-05 
***********************************************************
MR test results of  AD  on  Hair_Black : 
MR-APSS: beta =  0.014 , beta.se =  0.0207 , p-value =  4.9808e-01 . 
Total NO. of IVs=  69 , NO. of valid IVs with foreground signals:  17.15173 . 
***********************************************************
   user  system elapsed 
  1.434   0.899   1.166 
Run MR-APSS (Omega = 0), i.e. MR-APSS not accounting for polygenicity and pleiotropy. ... 
IV selection threshold: 5e-05 
***********************************************************
MR test results of 

...

***********************************************************
MR test results of  SWB  on  Hair_Black : 
MR-APSS: beta =  0.0312 , beta.se =  0.0422 , p-value =  4.5941e-01 . 
Total NO. of IVs=  65 , NO. of valid IVs with foreground signals:  64.9935 . 
***********************************************************
   user  system elapsed 
  1.695   1.009   1.352 
Pair:  SWB ~ Hair_Blonde 
Run MR-APSS ... 
IV selection threshold: 5e-05 
***********************************************************
MR test results of  SWB  on  Hair_Blonde : 
MR-APSS: beta =  11.4295 , beta.se =  11.2882 , p-value =  3.1129e-01 . 
Total NO. of IVs=  65 , NO. of valid IVs with foreground signals:  2.300813 . 
***********************************************************
   user  system elapsed 
 40.795  26.260  33.977 
Run MR-APSS (Omega = 0), i.e. MR-APSS not accounting for polygenicity and pleiotropy. ... 
IV selection threshold: 5e-05 
***********************************************************
MR test results 

Threshold = 1, the model will not correct for selection bias



***********************************************************
MR test results of  SWB  on  Hair_Blonde : 
MR-APSS: beta =  -0.0058 , beta.se =  0.0402 , p-value =  8.8488e-01 . 
Total NO. of IVs=  65 , NO. of valid IVs with foreground signals:  64.9935 . 
***********************************************************
   user  system elapsed 
  1.282   0.785   1.033 
Pair:  SWB ~ Tanning 
Run MR-APSS ... 
IV selection threshold: 5e-05 
***********************************************************
MR test results of  SWB  on  Tanning : 
MR-APSS: beta =  -0.1555 , beta.se =  0.2976 , p-value =  6.0137e-01 . 
Total NO. of IVs=  64 , NO. of valid IVs with foreground signals:  8.658149 . 
***********************************************************
   user  system elapsed 
  8.418   5.054   6.769 
Run MR-APSS (Omega = 0), i.e. MR-APSS not accounting for polygenicity and pleiotropy. ... 
IV selection threshold: 5e-05 
***********************************************************
MR test results of  SWB

Threshold = 1, the model will not correct for selection bias



***********************************************************
MR test results of  SWB  on  Tanning : 
MR-APSS: beta =  -0.0333 , beta.se =  0.0433 , p-value =  4.4251e-01 . 
Total NO. of IVs=  64 , NO. of valid IVs with foreground signals:  63.9936 . 
***********************************************************
   user  system elapsed 
  0.305   0.156   0.233 
     user    system   elapsed 
10512.427  5442.447  9139.462 


In [21]:
apss_res = read.table("NC_MRAPSS.MRres", header = T)
colnames(apss_res) = c("exposure","outcome", "nsnp","pi","nvalid","sigma.sq","tau.sq","beta.hat","se","pval",
                        "Method", "cor.Threshold", "IVstrength", "Threshold")
head(apss_res)
write.table(apss_res, "NC_MRAPSS.MRres", quote=F, col.names = T, append = F,row.names = F)

,exposure,outcome,nsnp,pi,nvalid,sigma.sq,tau.sq,b,se,pval,Method,cor.Threshold,IVstrength,Threshold
,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
1,AD,Hair_Light_Brown,69,0.2476728,17.08942,2.783954e-05,4.378918e-10,-6.722757e-03,0.018745343,0.71987,MR-APSS,5e-05,0.01289015,5e-05
2,AD,Hair_Light_Brown,69,0.2563745,17.68984,2.695711e-05,4.393785e-10,-5.793170e-03,0.017510986,0.74077,MR-APSS(Omega=0),5e-05,0.01313690,5e-05
3,AD,Hair_Light_Brown,69,0.2836725,19.57340,2.521091e-05,4.427412e-10,-8.142519e-03,0.016567372,0.62309,MR-APSS(C=I),5e-05,0.01374133,5e-05
4,AD,Hair_Light_Brown,69,0.9999000,68.99310,3.253142e-05,2.761819e-10,-1.039444e-03,0.011615897,0.92870,MR-APSS(Cor.SelectionBia=F),5e-05,0.03103007,5e-05
5,AD,Hair_Dark_Brown,69,0.2470616,17.04725,2.787310e-05,4.372200e-10,-1.329731e-05,0.004386458,0.99758,MR-APSS,5e-05,0.01287394,5e-05
6,AD,Hair_Dark_Brown,69,0.2558325,17.65244,2.701280e-05,8.829424e-10,8.722353e-04,0.017557966,0.96038,MR-APSS(Omega=0),5e-05,0.01312450,5e-05


# Run MR-APSS changing the IV threshold

In [5]:
# All the results will be saved to the file "NC_MRAPSS.MRres"
start = proc.time()
ts1 = c("AD", "ASD", "Daytime_Sleepiness", "Height_UKB",  "Intelligence", "RA",      
        "T2D", "Alcohol", "BMI", "Depression", "IBD", "MDD", "SCZ", "Angina", 
        "CAD", "HBP", "Income", "NEB", "Smoking", "Urate", "Anorexia", 
        "CD", "Height_GIANT", "Insomnia", "Neuroticism", "SWB")

ts2 = c("Hair_Light_Brown", "Hair_Dark_Brown",  "Hair_Black",    "Hair_Blonde",  "Tanning")

IV.Threshold = c(5e-06,5e-07, 5e-08)

for( exposure in ts1){
  
  for( outcome in ts2){
    
    cat("Pair: ", exposure,"~", outcome,"\n")
    # read in GWAS summary data for IVs
    clumped = try(read.table(paste0("./MRdat/", exposure,"~",outcome), header = T))
    
    # read in background parameters Omega and C
    C = try(as.matrix(read.table(paste0("./pairs_bg_paras/",exposure, "~", outcome,"_C"),
                                 header = F)))
    
    C = matrix(C[nrow(C), ], 2,2)
    
    Omega = try(as.matrix(read.table(paste0("./pairs_bg_paras/", exposure, "~", outcome,"_Omega"),
                                     header = F)))
      
    Omega = matrix(Omega[nrow(Omega), ], 2, 2)
    
    if(inherits(clumped , 'try-error')) clumped = NULL
    if(inherits(C, 'try-error')) next
    if(inherits(Omega, 'try-error')) next
    if(nrow(clumped) < 4 ) next
    
    # The p-value threshold for selection bias correction
    Threshold =  ifelse(IV.Threshold==5e-05, unique(clumped$Threshold), IV.Threshold)
    
    # MR-APSS
    cat("Run MR-APSS ... \n")
    res = run_APSS_func(clumped = clumped,
                        exposure = exposure,
                        outcome = outcome,
                        C = C,
                        Omega=Omega,
                        IV.Threshold = IV.Threshold,
                        Threshold = Threshold,
                        Cor.SelectionBias = T)
    
     
     # MR-APSS not accounting for selection bias
     cat("Run MR-APSS not accounting for selection bias ... \n")
     res_selec0 = run_APSS_func(clumped = clumped,
                            exposure = exposure,
                            outcome = outcome,
                            C = C,
                            Omega = Omega,
                            IV.Threshold = IV.Threshold,
                            Threshold = Threshold,
                            Cor.SelectionBias = F)
     res_selec0$method = "MR-APSS(Cor.SelectionBia=F)"
     
#      # saving resuts
     write.table(res, "NC_MRAPSS.MRres", quote=F, col.names = F, append = T,row.names = F)
     
     write.table(res_selec0, "NC_MRAPSS.MRres", quote=F, col.names = F, append = T,row.names = F)
     
}
  
}

print(proc.time()-start)

Pair:  AD ~ Hair_Light_Brown 
Run MR-APSS ... 
IV selection threshold: 5e-06 
***********************************************************
MR test results of  AD  on  Hair_Light_Brown : 
MR-APSS: beta =  -0.0065 , beta.se =  0.0194 , p-value =  7.3711e-01 . 
Total NO. of IVs=  24 , NO. of valid IVs with foreground signals:  14.10276 . 
***********************************************************
   user  system elapsed 
  0.647   0.206   0.428 
IV selection threshold: 5e-07 
***********************************************************
MR test results of  AD  on  Hair_Light_Brown : 
MR-APSS: beta =  -9e-04 , beta.se =  0.0202 , p-value =  9.6452e-01 . 
Total NO. of IVs=  16 , NO. of valid IVs with foreground signals:  11.26786 . 
***********************************************************
   user  system elapsed 
 10.843   3.212   7.029 
IV selection threshold: 5e-08 
***********************************************************
MR test results of  AD  on  Hair_Light_Brown : 
MR-APSS: beta =

Threshold = 1, the model will not correct for selection bias



***********************************************************
MR test results of  AD  on  Hair_Light_Brown : 
MR-APSS: beta =  -2e-04 , beta.se =  0.0153 , p-value =  9.8809e-01 . 
Total NO. of IVs=  24 , NO. of valid IVs with foreground signals:  23.9976 . 
***********************************************************
   user  system elapsed 
  3.457   0.944   2.201 
IV selection threshold: 5e-07 


Threshold = 1, the model will not correct for selection bias



***********************************************************
MR test results of  AD  on  Hair_Light_Brown : 
MR-APSS: beta =  -0.0114 , beta.se =  0.0171 , p-value =  5.0541e-01 . 
Total NO. of IVs=  16 , NO. of valid IVs with foreground signals:  15.9984 . 
***********************************************************
   user  system elapsed 
 13.594   4.219   8.909 
IV selection threshold: 5e-08 


...

***********************************************************
MR test results of  SWB  on  Hair_Black : 
MR-APSS: beta =  0.0621 , beta.se =  0.0746 , p-value =  4.0524e-01 . 
Total NO. of IVs=  13 , NO. of valid IVs with foreground signals:  12.9987 . 
***********************************************************
   user  system elapsed 
 16.245   4.982  10.615 
Pair:  SWB ~ Hair_Blonde 
Run MR-APSS ... 
IV selection threshold: 5e-06 
***********************************************************
MR test results of  SWB  on  Hair_Blonde : 
MR-APSS: beta =  0.2583 , beta.se =  0.2531 , p-value =  3.0756e-01 . 
Total NO. of IVs=  13 , NO. of valid IVs with foreground signals:  12.99802 . 
***********************************************************
   user  system elapsed 
 12.243   3.713   7.979 
Run MR-APSS not accounting for selection bias ... 
IV selection threshold: 5e-06 


Threshold = 1, the model will not correct for selection bias



***********************************************************
MR test results of  SWB  on  Hair_Blonde : 
MR-APSS: beta =  0.0749 , beta.se =  0.08 , p-value =  3.4921e-01 . 
Total NO. of IVs=  13 , NO. of valid IVs with foreground signals:  12.9987 . 
***********************************************************
   user  system elapsed 
  1.733   0.519   1.126 
Pair:  SWB ~ Tanning 
Run MR-APSS ... 
IV selection threshold: 5e-06 
***********************************************************
MR test results of  SWB  on  Tanning : 
MR-APSS: beta =  -0.1843 , beta.se =  0.2729 , p-value =  4.9946e-01 . 
Total NO. of IVs=  13 , NO. of valid IVs with foreground signals:  11.46728 . 
***********************************************************
   user  system elapsed 
 11.195   3.095   7.147 
Run MR-APSS not accounting for selection bias ... 
IV selection threshold: 5e-06 


Threshold = 1, the model will not correct for selection bias



***********************************************************
MR test results of  SWB  on  Tanning : 
MR-APSS: beta =  -0.0692 , beta.se =  0.0851 , p-value =  4.1658e-01 . 
Total NO. of IVs=  13 , NO. of valid IVs with foreground signals:  12.9987 . 
***********************************************************
   user  system elapsed 
  0.096   0.035   0.066 
    user   system  elapsed 
8640.571 2413.607 5848.112 
